# Exploring Grib Files

In [1]:
import iris
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import os
import pygrib
from mpl_toolkits.basemap import Basemap
plt.interactive(True)

KeyError: 'PROJ_LIB'

## Load data

In [8]:
grib_files = [os.path.join("data", f) for f in os.listdir("data")]

In [9]:
cubes = iris.load(grib_files[0])

In [31]:
cube_raw = iris.load_raw(grib_files[0])

In [11]:
print(cubes)

0: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
1: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
2: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
3: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
4: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
5: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
6: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
7: unknown / (unknown)                 (time: 61; latitude: 123; longitude: 155)
8: wind_from_direction / (degrees)     (time: 61; latitude: 123; longitude: 155)
9: wind_speed / (m s-1)                (time: 61; latitude: 123; longitude: 155)
10: x_wind / (m s-1)                    (time: 61; latitude: 123; longitude: 155)
11: y_wind / (m s-1)                    (time: 61; latitude: 123; longitude: 155)


In [43]:
cubes[0].name()

'unknown'

In [21]:
wind_speed = cubes[9]

In [22]:
print(wind_speed)

wind_speed / (m s-1)                (time: 61; latitude: 123; longitude: 155)
     Dimension coordinates:
          time                           x             -               -
          latitude                       -             x               -
          longitude                      -             -               x
     Auxiliary coordinates:
          forecast_period                x             -               -
     Scalar coordinates:
          forecast_reference_time: 2018-12-06 00:00:00


# With PyGrib

In [45]:
grbs = pygrib.open(grib_files[0])

In [49]:
grb = grbs.read(1)[0]
grb

1:Wind speed:m s**-1 (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201812060000

In [54]:
data = grb.values
lat, lon = grb.latlons()

In [55]:
data

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, 16.48, 16.42, ..., 4.11, 3.34, --],
        [--, 16.26, 16.1, ..., 4.98, 4.3100000000000005, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True, False, False, ..., False, False,  True],
        [ True, False, False, ..., False, False,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=9999.0)